## Generating fake EHR data
by: Sparkle Russell-Puleri and Dorian Puleri

For the purpose of this three part tutorial, we generated some artificial EHR data to demonstrate how EHR data should be processed for use in sequence models. Please note that this data has no clinical relevance and was just created for training purposes only.

In [1]:
import pandas as pd
import numpy as np

### Patient Admission Table
This table contains information on the patient admission history and times. The features generated were:
1. `PatientID`- Unique identifier that stay with the patient permanently
2. `Admission ID` - Specific to each visit
3. `AdmissionStartDate` - Date and time of admission 
4. `AdmissionEndDate` - Date and time of discharge after care for a specific admission ID

In [84]:
admission_table = {'Patient 1': {'PatientID':'A1234-B456', 
                          'Admission ID':[12,34,15], 
                          'AdmissionStartDate':['2019-01-03 9:34:55','2019-02-03 10:50:55','2019-04-03 12:34:55'],
                          'AdmissionEndDate':['2019-01-07 8:45:43','2019-03-04 1:50:32','2019-04-03 5:38:18']},
                   'Patient 2': {'PatientID':'B1234-C456', 
                          'Admission ID':[13,34], 
                          'AdmissionStartDate':['2018-01-03 9:34:55','2018-02-03 10:50:55'],
                          'AdmissionEndDate':['2018-01-07 8:45:43','2018-03-04 1:50:32']}}
admission_table = (pd.concat({k: pd.DataFrame(v) for k, v in admission_table.items()}).reset_index(level=1, drop=True))
admission_table = admission_table.reset_index(drop=True)

### Patient Diagnosis Table
The diagnosis table is quite unique, as it can contain several diagnosis codes for the same visit. For example, Patient 1 was diagnosised with diabetes (`PrimaryDiagnosisCode`:E11.64) during his/her first visit (`Admission ID`:12). However, this code also shows up on subsequent visits (`Admission ID`:34, 15), why is that? Well if a patient is diagnosised with an uncurable condition he/she that code will always be associated all subsequent visits. On the other hand, codes associated with acute care, will come and go as seen with `PrimaryDiagnosisCode`:780.96(Headache). 

In [85]:
Patient_1 = {'PatientID':'A1234-B456', 
             'Admission ID':[12,34,15], 
             'PrimaryDiagnosisCode':[['E11.64','I25.812','I25.10'],
                                     ['E11.64','I25.812','I25.10','780.96','784.0'],
                                     ['E11.64','I25.812','I25.10','786.50','401.9','789.00']],
             'CodingSystem':['ICD-9','ICD-9','ICD-9'],
             'DiagnosisCodeDescription':[['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris'],
                                         ['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris',
                                          'Generalized Pain', 'Dizziness and giddiness'],
                                         ['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris',
                                          'Chest pain, unspecified','Essential hypertension, unspecified',
                                          'Abdominal pain, unspecified site']]}
Patient_2 = {'PatientID':'B1234-C456', 
              'Admission ID':[13,34], 
              'PrimaryDiagnosisCode':[['M05.59','Z13.85','O99.35'],['M05.59','Z13.85','O99.35','D37.0']],
              'CodingSystem':['ICD-9','ICD-9'],
              'DiagnosisCodeDescription':[['Rheumatoid polyneuropathy with rheumatoid arthritis of multiple sites',
                                           'Encounter for screening for nervous system disorders',
                                           'Diseases of the nervous system complicating pregnancy, childbirth, and the puerperium'],
                                          ['Rheumatoid polyneuropathy with rheumatoid arthritis of multiple sites',
                                           'Encounter for screening for nervous system disorders',
                                           'Diseases of the nervous system complicating pregnancy, childbirth, and the puerperium',
                                           'Neoplasm of uncertain behavior of lip, oral cavity and pharynx']]}

### Helper functions for parsing data from a dictionary to DataFrame

In [86]:
def process_ehr(Patient1,Patient2):
    pt_diagnosis_table = [Patient1,Patient2]
    pt_diagnosis_table = pd.concat([pd.DataFrame({k:v for k,v in d.items()}) for d in pt_diagnosis_table])
    
    pt_diagnosis_table = (pt_diagnosis_table.set_index(['PatientID', 'Admission ID','CodingSystem'])
              .apply(lambda x: x.apply(pd.Series).stack())
              .reset_index()
              .drop('level_3', 1))
    return pt_diagnosis_table
def hash_key(df):
    df['HashKey'] = df['PatientID'].\
    apply(lambda x: x.split('-')[0]) + '-' + df['Admission ID'].astype('str')
    cols = [df.columns[-1]] + [col for col in df if col != df.columns[-1]]
    print(cols)
    return df[cols]

In [87]:
diagnosis_table = process_ehr(Patient_1,Patient_2)
diagnosis_table.head()

,PatientID,Admission ID,CodingSystem,PrimaryDiagnosisCode,DiagnosisCodeDescription
0,A1234-B456,12,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
1,A1234-B456,12,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...
2,A1234-B456,12,ICD-9,I25.10,Atherosclerotic heart disease of native corona...
3,A1234-B456,34,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
4,A1234-B456,34,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...


## Create a hashkey for Admission ID
Why do this step? Unless your EHR system has uniqely identifiable Admission IDs for each patients visit, it would be difficult to associate each patient ID with a unique `Admission ID`. To demonstrate this, we deliberately created double digit `Admission ID`s one of which was repeated ( `Admission ID`: 34) for both patients. To avoid this, we took a pre-cautionary step to create a hash key that is a unique combination of the first half of the the unique `PatientID` hyphenated with the patient's specific `Admission ID`.

In [88]:
diagnosis_table[diagnosis_table['Admission ID']==34]

,PatientID,Admission ID,CodingSystem,PrimaryDiagnosisCode,DiagnosisCodeDescription
3,A1234-B456,34,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
4,A1234-B456,34,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...
5,A1234-B456,34,ICD-9,I25.10,Atherosclerotic heart disease of native corona...
6,A1234-B456,34,ICD-9,780.96,Generalized Pain
7,A1234-B456,34,ICD-9,784.0,Dizziness and giddiness
17,B1234-C456,34,ICD-9,M05.59,Rheumatoid polyneuropathy with rheumatoid arth...
18,B1234-C456,34,ICD-9,Z13.85,Encounter for screening for nervous system dis...
19,B1234-C456,34,ICD-9,O99.35,Diseases of the nervous system complicating pr...
20,B1234-C456,34,ICD-9,D37.0,"Neoplasm of uncertain behavior of lip, oral ca..."


In [89]:
# Diag
diagnosis_table = hash_key(diagnosis_table)
admission_table = hash_key(admission_table)

['HashKey', 'PatientID', 'Admission ID', 'CodingSystem', 'PrimaryDiagnosisCode', 'DiagnosisCodeDescription']
['HashKey', 'PatientID', 'Admission ID', 'AdmissionStartDate', 'AdmissionEndDate']


## Final Admission and Diagnosis Tables generated with fake EHR data

In [90]:
admission_table.head()

,HashKey,PatientID,Admission ID,AdmissionStartDate,AdmissionEndDate
0,A1234-12,A1234-B456,12,2019-01-03 9:34:55,2019-01-07 8:45:43
1,A1234-34,A1234-B456,34,2019-02-03 10:50:55,2019-03-04 1:50:32
2,A1234-15,A1234-B456,15,2019-04-03 12:34:55,2019-04-03 5:38:18
3,B1234-13,B1234-C456,13,2018-01-03 9:34:55,2018-01-07 8:45:43
4,B1234-34,B1234-C456,34,2018-02-03 10:50:55,2018-03-04 1:50:32


### Cast Admission dates to Date time object 

In [91]:
admission_table[['AdmissionStartDate','AdmissionEndDate']] = admission_table[['AdmissionStartDate','AdmissionEndDate']]\
                .apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
admission_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
HashKey               5 non-null object
PatientID             5 non-null object
Admission ID          5 non-null int64
AdmissionStartDate    5 non-null datetime64[ns]
AdmissionEndDate      5 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 280.0+ bytes


In [93]:
diagnosis_table.head()

,HashKey,PatientID,Admission ID,CodingSystem,PrimaryDiagnosisCode,DiagnosisCodeDescription
0,A1234-12,A1234-B456,12,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
1,A1234-12,A1234-B456,12,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...
2,A1234-12,A1234-B456,12,ICD-9,I25.10,Atherosclerotic heart disease of native corona...
3,A1234-34,A1234-B456,34,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
4,A1234-34,A1234-B456,34,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...


### Write tables to csv

In [94]:
%%bash
mkdir data   

mkdir: data: File exists


In [95]:
# Write files to data directory
diagnosis_table.to_csv('data/Diagnosis_Table.csv',encoding='UTF-8',index=False)
admission_table.to_csv('data/Admissions_Table.csv',encoding='UTF-8',index=False,date_format='%Y-%m-%d')

## Next:
We will process the data in preparation for modeling